# Semantinis branduolys su OpenBnB MCP serverio integracija

Šiame užrašų knygelėje parodoma, kaip naudoti Semantinį branduolį su tikruoju OpenBnB MCP serveriu, norint ieškoti realių Airbnb apgyvendinimo vietų naudojant MCPStdioPlugin. LLM prieigai naudojama Azure AI Foundry. Norėdami nustatyti savo aplinkos kintamuosius, galite vadovautis [Nustatymo pamoka](/00-course-setup/README.md)


## Importuokite Reikalingus Paketus


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Sukurti MCP įskiepio ryšį

Prisijungsime prie [OpenBnB MCP serverio](https://github.com/openbnb-org/mcp-server-airbnb) naudodami MCPStdioPlugin. Šis serveris suteikia Airbnb paieškos funkcionalumą per @openbnb/mcp-server-airbnb paketą.


## Kliento kūrimas

Šiame pavyzdyje naudosime Azure AI Foundry, kad pasiektume LLM. Įsitikinkite, kad jūsų aplinkos kintamieji yra tinkamai nustatyti.


## Aplinkos konfigūracija

Konfigūruokite Azure OpenAI nustatymus. Įsitikinkite, kad turite nustatytus šiuos aplinkos kintamuosius:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Suprasti OpenBnB MCP integraciją

Ši užrašų knygelė jungiasi prie **tikro OpenBnB MCP serverio**, kuris teikia tikrą Airbnb paieškos funkcionalumą.

### Kaip tai veikia:

1. **MCPStdioPlugin**: Naudoja standartinę įvesties/išvesties komunikaciją su MCP serveriu
2. **Tikras NPM paketas**: Atsisiunčia ir paleidžia `@openbnb/mcp-server-airbnb` per npx
3. **Tiesioginiai duomenys**: Grąžina tikrus Airbnb nekilnojamojo turto duomenis iš jų API
4. **Funkcijų atradimas**: Agentas automatiškai aptinka galimas funkcijas iš MCP serverio

### Galimos funkcijos:

OpenBnB MCP serveris paprastai teikia:
- **search_listings** - Ieškoti Airbnb nekilnojamojo turto pagal vietą ir kriterijus
- **get_listing_details** - Gauti išsamią informaciją apie konkrečius objektus
- **check_availability** - Patikrinti prieinamumą konkrečioms datoms
- **get_reviews** - Gauti atsiliepimus apie objektus
- **get_host_info** - Gauti informaciją apie objektų šeimininkus

### Būtinos sąlygos:

- **Node.js** įdiegta jūsų sistemoje
- **Interneto ryšys** MCP serverio paketo atsisiuntimui
- **NPX** prieinamas (yra kartu su Node.js)

### Ryšio testavimas:

MCP serverį galite patikrinti rankiniu būdu paleisdami:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Tai atsisiųs ir paleis OpenBnB MCP serverį, prie kurio Semantic Kernel prisijungs, kad gautų tikrus Airbnb duomenis.


## Paleisti agentą su OpenBnB MCP serveriu

Dabar paleisime AI agentą, kuris jungiasi prie OpenBnB MCP serverio, kad ieškotų tikrų Airbnb apgyvendinimo vietų Stokholme 2 suaugusiems ir 1 vaikui. Galite laisvai pakeisti `user_inputs` sąrašą, kad modifikuotumėte paieškos kriterijus.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Santrauka  
Sveikiname! Jūs sėkmingai sukūrėte AI agentą, kuris integruojasi su realaus pasaulio apgyvendinimo paieška naudodamas Model Context Protocol (MCP):

Naudotos technologijos:  
Semantic Kernel - intelektualių agentų kūrimui su Azure OpenAI  
Azure AI Foundry - LLM galimybėms ir pokalbių užbaigimui  
MCP (Model Context Protocol) - standartizuotai įrankių integracijai  
OpenBnB MCP Server - realiai Airbnb paieškos funkcionalumui  
Node.js/NPX - išoriniam MCP serveriui paleisti  

Ką išmokote:  
MCP integracija: Semantic Kernel agentų prijungimas prie išorinių MCP serverių  
Duomenų prieiga realiu laiku: Tikrų Airbnb objektų paieška per gyvas API  
Protokolo komunikacija: stdio komunikacijos naudojimas tarp agento ir MCP serverio  
Funkcijų atradimas: Automatinis galimų funkcijų aptikimas iš MCP serverių  
Atsakymų srautinimas: Funkcijų iškvietimų fiksavimas ir registravimas realiu laiku  
HTML atvaizdavimas: Agentų atsakymų formatavimas stilizuotomis lentelėmis ir interaktyviais vaizdais  

Kiti žingsniai:  
Integruoti papildomus MCP serverius (orai, skrydžiai, restoranai)  
Sukurti daugiagentę sistemą, derinant MCP ir A2A protokolus  
Sukurti individualius MCP serverius savo duomenų šaltiniams  
Įgyvendinti nuolatinę pokalbių atmintį tarp sesijų  
Patalpinti agentą Azure Functions su MCP serverių orkestracija  
Pridėti vartotojų autentifikaciją ir rezervavimo galimybes  

Pagrindiniai MCP architektūros privalumai:  
Standartizacija: Universalus protokolas AI agentų prijungimui prie išorinių įrankių  
Duomenys realiu laiku: Prieiga prie gyvos, naujausios informacijos iš įvairių paslaugų  
Išplečiamumas: Lengva integruoti naujus duomenų šaltinius ir įrankius  
Sąveikumas: Veikia su skirtingais AI pagrindais ir agentų platformomis  
Atskirų funkcijų atskyrimas: Aiškus AI logikos ir išorinių duomenų prieigos atskyrimas  



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
